In [1]:
import torch
torch.__version__

'1.8.0'

In [2]:
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
from torch.nn.parallel import DistributedDataParallel as DDP

In [19]:
def example(rank, world_size):
    dist.init_process_group("gloo", rank=rank, world_size = world_size)
    model = nn.Linear(10, 10).to(rank)
    ddp_model = DDP(model, device_ids=[rank])
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.001)
    
    outputs = ddp_model(torch.randn(20, 10).to(rank))
    labels = torch.randn(20, 10).to(rank)
    loss_fn(outputs, labels).backward()
    optimizer.step()
    
def ddp_main():
    world_size = 2
    rank='cuda'
    mp.spawn(example,
            args=(world_size,),
            nprocs=world_size,
            join=True)
    

In [20]:
import os
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '29500'
ddp_main()

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/nfs/admin0/lewisliu/.conda/envs/transformer/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/home/nfs/admin0/lewisliu/.conda/envs/transformer/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'example' on <module '__main__' (built-in)>


ProcessExitedException: process 0 terminated with exit code 1

In [18]:
torch.cuda.device_count()

2

In [ ]:
test